# code for make bed file to peak distribution

This function is uesd to generate the peak distribution from peak.bed



### Input 
The input of the code is bed files path, and you could use mclapply to calculate parallelly.

### output 

The output of the code is a list with 24 elements, each element is a dataframe. This dataframe contains "site" and "value" columns, "site" is the position of the peak region, and the "value" is always 1 because we only consider hitted positions in one bed.



**Required:**

"path" : the bed to be transfered, it could contained many  bed files;

"savepath": the folder that you want to save the trasfered peak distribution






## import and functions

In [7]:
library(Seurat)
library(dplyr)
library(parallel)
library(purrr)
options(scipen=200)

In [2]:
idx2list = function(x){
    return(x[[1]]:(as.numeric(x[[2]])-1))
}

In [3]:
chr2site= function(feature_list){
    
    index = strsplit(feature_list,split='-')
    site = mclapply(index,idx2list,mc.cores = 40 )
    site = unlist(site)
    site = unique(site)
    value = rep(1,length(site))
    df = cbind(site,value)
    return(data.frame(df))
}

In [4]:
bed2site = function(bed){
    
    f_chr = mclapply(bed,strsplit,split=':')
    f_chr = matrix(unlist(f_chr), ncol=2,byrow=T)
    f_chr = aggregate(f_chr, by=list(f_chr[,1]), FUN=paste,sep=',')[,-2]
    row.names(f_chr) = f_chr[,1]
    f_chr = f_chr[chrlist,2]
    res = mclapply(f_chr,chr2site,mc.cores=24)
    names(res) = chrlist
    return (res)
}

In [5]:
file2site = function(s){
    data = read.table(paste0(path,subpath[s]))#gzfile if file is gz
    feature = paste0(data[,1],':',data[,2],'-',data[,3])
    feature = unique(feature)
    chr_all = bed2site(feature)
    saveRDS(chr_all,paste0(savepath,subpath3[[s]],'.rds'))
    return(1)
}

## use code to generate

## ENCODE part

In [96]:
path = '/nfs/mqc/Consensus_peak/data/Zhang_2021_Cell_AdultTissues/others/bed_celltype/union/'
savepath= '/nfs/mqc/Consensus_peak/data/Zhang_2021_Cell_AdultTissues/others/peak_distribution_celltype/union/'
subpath = dir(path)

chrlist = append(paste0('chr',1:22),c('chrX','chrY'))

res = mclapply(1:length(subpath),file2site,mc.cores=24)

## ATACdb part